In [10]:
import numpy as np
import pandas as pd
import torch
from torch.nn import Module
from torch.nn import Sequential
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import AvgPool2d
from torch.nn import BatchNorm2d
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Tanh
from torch.nn import ELU
from torch.nn import LogSoftmax
from torch.nn import Dropout
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import flatten
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms.autoaugment import AutoAugmentPolicy
from torch.utils.data.sampler import SubsetRandomSampler
# from dataset import load_dataset

from tqdm.notebook import tqdm

from torchsummary import summary
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [11]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CNN(Module):
  def __init__(self, in_channels, classes):
    super(CNN, self).__init__()

    self.classes = classes

    self.conv_block_1 = Sequential(
        Conv2d(in_channels = in_channels, out_channels = 64,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(64),
        # Tanh(),
        ReLU(),
        Conv2d(in_channels = 64, out_channels = 64,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(64),
        ReLU(),
        MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
    )
    self.conv_block_2 = Sequential(
        Conv2d(in_channels = 64, out_channels = 128,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(128),
        ReLU(),
        Conv2d(in_channels = 128, out_channels = 128,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(128),
        ReLU(),
        MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
    )

    self.conv_block_3 = Sequential(
        Conv2d(in_channels = 128, out_channels = 256,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(256),
        ReLU(),
        Conv2d(in_channels = 256, out_channels = 256,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(256),
        ReLU(),
        MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
    )


    self.conv_block_4 = Sequential(
        Conv2d(in_channels = 256, out_channels = 512,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(512),
        ReLU(),
        Conv2d(in_channels = 512, out_channels = 512,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(512),
        ReLU(),
        MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
    )

    # self.conv_block_5 = Sequential(
    #     Conv2d(in_channels = 512, out_channels = 512,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
    #     BatchNorm2d(512),
    #     ReLU(),
    #     Conv2d(in_channels = 512, out_channels = 512,kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
    #     BatchNorm2d(512),
    #     ReLU(),
    #     MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
    # )

    # self.fc_block = Sequential(
    #     Linear(in_features = 7 * 7 * 512, out_features = 4096),
    #     ReLU(),
    #     Dropout(0.4),
    #     Linear(in_features = 4096, out_features = 4096),
    #     ReLU(),
    #     Dropout(0.4),
    #     Linear(in_features = 4096, out_features = self.classes)
    # )
    # self.fc_block = Sequential(
    # nn.Flatten(),
    # nn.Linear(2048, 512),  # Adjust input size to match the output size after flattening
    # nn.ReLU(),
    # nn.Dropout(0.5),
    # nn.Linear(512, self.classes))

    # self.fc_block = Sequential(
    # nn.Flatten(),
    # nn.Linear(2048, 512),  # Adjust input size to match the output size after flattening
    # nn.ReLU(),
    # nn.Dropout(0.5),
    # nn.Linear(512, self.classes))
    self.fc_block = Sequential(
    nn.Flatten(),
    nn.Linear(512*4, 512),  # Adjust input size to match the output size after flattening
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),  # Adding a second fully connected layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, self.classes)  # Output layer
)


  def forward(self, x):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.conv_block_3(x)
    print(x.shape)
    x = self.conv_block_4(x)
    print(x.shape)
    x = self.conv_block_5(x)
    print(x.shape)
    # x = self.conv_block_5(x)
    x = flatten(x, 1)
    print(x.shape)
    x = self.fc_block(x)
    # print(x.shape)
    out = LogSoftmax(x)
    # print(x.shape)
    return x




In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to range [-1, 1]
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize the model

num_classes=10
model = CNN(3, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
# Define optimizer with momentum
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # print(images.size())
        optimizer.zero_grad()
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        #backward pass
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")
#using tanh....
# Files already downloaded and verified
# Files already downloaded and verified
# Epoch 1/10, Loss: 1.7178710360661187
# Epoch 2/10, Loss: 1.3661400325158064
# Epoch 3/10, Loss: 1.1718245865134023
# Epoch 4/10, Loss: 1.0442841635335742
# Epoch 5/10, Loss: 0.9434976887215129
# Epoch 6/10, Loss: 0.834720287367206
# Epoch 7/10, Loss: 0.7555759236254656
# Epoch 8/10, Loss: 0.684088701177436
# Epoch 9/10, Loss: 0.6128920804128013
# Epoch 10/10, Loss: 0.5536008727215135
# Accuracy on test set: 77.72%
# Epoch 1/10, Loss: 1.5861312919260595
# Epoch 2/10, Loss: 1.1503502731128117
# Epoch 3/10, Loss: 0.9183450589323288
# Epoch 4/10, Loss: 0.7766158564392563
# Epoch 5/10, Loss: 0.6582114441163095
# Epoch 6/10, Loss: 0.5690752352061479
# Epoch 7/10, Loss: 0.5033227935471498
# Epoch 8/10, Loss: 0.43710428074269037
# Epoch 9/10, Loss: 0.3813700695019549
# Epoch 10/10, Loss: 0.3301420951133494
# Accuracy on test set: 76.56%
# Epoch 1/10, Loss: 1.6338122188282744
# Epoch 2/10, Loss: 1.1588779518671353
# Epoch 3/10, Loss: 0.910141977812628
# Epoch 4/10, Loss: 0.7591500263613509
# Epoch 5/10, Loss: 0.6384064620908569
# Epoch 6/10, Loss: 0.5442136704845502
# Epoch 7/10, Loss: 0.4667827045292501
# Epoch 8/10, Loss: 0.4111713199588039
# Epoch 9/10, Loss: 0.35128747323132536
# Epoch 10/10, Loss: 0.2963946273888621
# Accuracy on test set: 85.5%
# Epoch 1/10, Loss: 1.5588648686628512
# Epoch 2/10, Loss: 1.0070512165575076
# Epoch 3/10, Loss: 0.761334937132533
# Epoch 4/10, Loss: 0.6003601265228008
# Epoch 5/10, Loss: 0.474278630960323
# Epoch 6/10, Loss: 0.3781060623028851
# Epoch 7/10, Loss: 0.299476518845924
# Epoch 8/10, Loss: 0.23537105634389327
# Epoch 9/10, Loss: 0.18608344889119688
# Epoch 10/10, Loss: 0.14970063536292147

# tanh, moment = 0

# Epoch 1/10, Loss: 2.0486078402575325
# Epoch 2/10, Loss: 1.7806983213595418
# Epoch 3/10, Loss: 1.586581483521425
# Epoch 4/10, Loss: 1.4680592153993104
# Epoch 5/10, Loss: 1.3802319844360547
# Epoch 6/10, Loss: 1.2950305208525694
# Epoch 7/10, Loss: 1.220568087094885
# Epoch 8/10, Loss: 1.1467510557845426
# Epoch 9/10, Loss: 1.0860859493312933
# Epoch 10/10, Loss: 1.0297946633432833
# Accuracy on test set: 57.37%

# Sigmoid, moment = 0

# Epoch 1/10, Loss: 2.346648204357118
# Epoch 2/10, Loss: 2.340106597032084
# Epoch 3/10, Loss: 2.33679483552723
# Epoch 4/10, Loss: 2.3336031391187704
# Epoch 5/10, Loss: 2.3349944121392485
# Epoch 6/10, Loss: 2.330765844610951
# Epoch 7/10, Loss: 2.328891118774024
# Epoch 8/10, Loss: 2.3269583279519437
# Epoch 9/10, Loss: 2.325028521020699
# Epoch 10/10, Loss: 2.325691189607391
# Accuracy on test set: 10.0%

Epoch 1/10, Loss: 1.7178710360661187
Epoch 2/10, Loss: 1.3661400325158064
Epoch 3/10, Loss: 1.1718245865134023
Epoch 4/10, Loss: 1.0442841635335742
Epoch 5/10, Loss: 0.9434976887215129
Epoch 6/10, Loss: 0.834720287367206
Epoch 7/10, Loss: 0.7555759236254656
Epoch 8/10, Loss: 0.684088701177436
Epoch 9/10, Loss: 0.6128920804128013
Epoch 10/10, Loss: 0.5536008727215135
Accuracy on test set: 77.72%


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # print(images.size())
        optimizer.zero_grad()
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        #backward pass
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")
#using sigmoid...

Epoch 1/10, Loss: 1.5861312919260595
Epoch 2/10, Loss: 1.1503502731128117
Epoch 3/10, Loss: 0.9183450589323288
Epoch 4/10, Loss: 0.7766158564392563
Epoch 5/10, Loss: 0.6582114441163095
Epoch 6/10, Loss: 0.5690752352061479
Epoch 7/10, Loss: 0.5033227935471498
Epoch 8/10, Loss: 0.43710428074269037
Epoch 9/10, Loss: 0.3813700695019549
Epoch 10/10, Loss: 0.3301420951133494
Accuracy on test set: 76.56%


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # print(images.size())
        optimizer.zero_grad()
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        #backward pass
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")
#using RelU

Epoch 1/10, Loss: 1.6338122188282744
Epoch 2/10, Loss: 1.1588779518671353
Epoch 3/10, Loss: 0.910141977812628
Epoch 4/10, Loss: 0.7591500263613509
Epoch 5/10, Loss: 0.6384064620908569
Epoch 6/10, Loss: 0.5442136704845502
Epoch 7/10, Loss: 0.4667827045292501
Epoch 8/10, Loss: 0.4111713199588039
Epoch 9/10, Loss: 0.35128747323132536
Epoch 10/10, Loss: 0.2963946273888621
Accuracy on test set: 85.5%


The above code was executed with ReLU as the activation function


In [13]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # print(images.size())
        optimizer.zero_grad()
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        #backward pass
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")
#using RelU

torch.Size([64, 256, 4, 4])
torch.Size([64, 512, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 512])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x512 and 2048x512)